# RAG 파이프라인 예시 - 엑셀 → Chroma → Retrieval QA

이 노트북에서는 엑셀 파일(`.xlsx`)로 관리되는 교통사고 사례 데이터를 로드한 뒤,
1. 텍스트 추출 및 청크화
2. 임베딩(벡터화)
3. Chroma DB에 저장
4. 간단한 Retrieval + LLM(QA) 시연
을 실행해 봅니다.


In [2]:
!pip install "langchain==0.2.6"
!pip install "ibm-watsonx-ai==1.0.10"
!pip install "langchain_ibm==0.1.8"
!pip install "langchain_community==0.2.6"
!pip install "sentence-transformers==3.0.1"
!pip install "chromadb==0.5.3"
!pip install "pydantic==2.8.2"
!pip install "langchain-huggingface==0.0.3"
!pip install "python-dotenv==1.0.1"

  Using cached chromadb-0.5.3-py3-none-any.whl.metadata (6.8 kB)
  Using cached chroma-hnswlib-0.7.3.tar.gz (31 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Using cached chromadb-0.5.3-py3-none-any.whl (559 kB)
  error: subprocess-exited-with-error
  
  × Building wheel for chroma-hnswlib (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [63 lines of output]
      running bdist_wheel
      running build
      running build_ext
      creating tmp
      g++ -fno-strict-overflow -Wsign-compare -DDYNAMIC_ANNOTATIONS_ENABLED=1 -DNDEBUG -O2 -fexceptions -g -grecord-gcc-switches -pipe -Wall -Werror=format-security -Wp,-D_FORTIFY_SOURCE=2 -Wp,-D_GLIBCXX_ASSERTIONS -fstack-protector-strong -m64 -march=x86-64-v2 -mtune=generic -fasynchronous-unwind-tables -fstack-clash-protection -fcf-protection -O2 -fexceptions -g -grecord-gcc-switches -pipe -Wall -Werror=format-security -Wp,-D_FORTIFY_

In [3]:
import os
import glob
import json
import re
import sys
import pandas as pd

!pip install pysqlite3-binary
__import__('pysqlite3')
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

from dotenv import load_dotenv

##################################
# LangChain & Watsonx imports
##################################
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.llms import WatsonxLLM
from langchain_core.prompts import ChatPromptTemplate
from langchain.docstore.document import Document
from langchain.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import DecodingMethods

##################################
# Watsonx Credentials
##################################
project_id = os.getenv("PROJECT_ID", None)
wml_credentials = {
    "apikey": os.getenv("API_KEY", None),
    "url": "https://us-south.ml.cloud.ibm.com"  # region url
}

In [4]:
load_dotenv()

True

In [5]:
def chunk_text(text, chunk_size=500, chunk_overlap=50):
    """
    RecursiveCharacterTextSplitter를 사용하여 텍스트를 청크로 분할.
    """
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
    )
    return splitter.split_text(text)

excel_file = "accident_data1.xlsx"  

# 1) 엑셀 로드
df = pd.read_excel(excel_file)
df.head()

,case_id,scenario,explanation,precedents,video,reference1,reference2,reference3,reference4,reference5,reference6,reference7,reference8,Unnamed: 13,Unnamed: 14
0,보1,"신호기가 있는 횡단보도에서 녹색신호에 교차로를 통과한 차량(직진, 좌회전, 우회전 ...",도로교통법 제5조에 따라 보행자나 차마는 신호기의 신호에 따라야 할 의무가 있으므로...,⊙대법원 1990.8.10. 선고 90도1116 판결 횡단보도의 표지판이나 신호대가...,https://drive.google.com/file/d/1MPFmVenWqLqjO...,도로교통법 제5조에 따라 보행자나 차마는 신호기의 신호에 따라야 할 의무가 있으므로...,① 야간 또는 주·정차된 차량 사이에서 보행자가 걸어 나오는 등 운전자가 보행자의 ...,⊙ 차량이 녹색신호에 교차로를 진입하여 보행자신호등 적색신호에 횡단보도를 건너고 있...,"⊙도로교통법 제5조(신호 또는 지시에 따를 의무)\n① 도로를 통행하는 보행자, 차...",⊙도로교통법 제27조(보행자의 보호)\n① 모든 차 또는 노면전차의 운전자는 보행자...,⊙도로교통법 제48조(안전운전 및 친환경 경제운전의 의무)\n① 모든 차 또는 노면...,NaN,⊙도로교통법 시행규칙 별표 2(신호기가 표시하는 신호의 종류 및 신호의 뜻),NaN,NaN
1,보2,"신호기가 있는 횡단보도에서 황색신호에 교차로를 통과한 차량(직진, 좌회전, 우회전 ...",차량이 황색신호에 진입한 경우 신호를 위반한 것이므로 보1 기준보다 보행자의 과실을...,⊙대법원 1990.8.10. 선고 90도1116 판결\n횡단보도의 표지판이나 신호대...,https://drive.google.com/file/d/1nsSoQyIPAaPy_...,차량이 황색신호에 진입한 경우 신호를 위반한 것이므로 보1 기준보다 보행자의 과실을...,① 야간 또는 주·정차된 차량 사이에서 보행자가 걸어 나오는 등 운전자가 보행자의 ...,NaN,"⊙도로교통법 제5조(신호 또는 지시에 따를 의무)\n① 도로를 통행하는 보행자, 차...",⊙도로교통법 제27조(보행자의 보호)\n① 모든 차 또는 노면전차의 운전자는 보행자...,⊙도로교통법 제48조(안전운전 및 친환경 경제운전의 의무)\n① 모든 차 또는 노면...,⊙도로교통법 시행령 제6조(경찰공무원을 보조하는 사람의 범위)\n법 제5조 제1항 ...,⊙도로교통법 시행규칙 별표 2(신호기가 표시하는 신호의 종류 및 신호의 뜻),NaN,NaN
2,보3,"신호기가 있는 횡단보도에서 황색신호에 교차로를 통과한 차량(직진, 좌회전)이 보행자...",도로교통법 제5조에 따라 보행자나 차마 등은 신호기의 신호에 따라야 할 의무가 있으...,x,https://drive.google.com/file/d/1TpcR-6A2uD6HR...,도로교통법 제5조에 따라 보행자나 차마 등은 신호기의 신호에 따라야 할 의무가 있으...,① 보행자가 보행신호등 녹색신호가 작동하자마자 횡단하는 경우로서 차량에게 예견 및 ...,녹색점멸신호는 녹색신호와 동일하게 본다(도로교통법 시행규칙 별표2 참조). 이하 동...,"⊙도로교통법 제5조(신호 또는 지시에 따를 의무)\n① 도로를 통행하는 보행자, 차...",⊙도로교통법 제27조(보행자의 보호)\n① 모든 차 또는 노면전차의 운전자는 보행자...,⊙도로교통법 제48조(안전운전 및 친환경 경제운전의 의무)\n① 모든 차 또는 노면...,⊙도로교통법 시행규칙 별표 2(신호기가 표시하는 신호의 종류 및 신호의 뜻),NaN,NaN,NaN
3,보4,"신호기가 있는 횡단보도에서 황색신호에 교차로에 통과한 차량(직진, 좌회전)이 보행자...",보행자가 녹색신호에 횡단을 개시하여 녹색신호에 충돌한 경우 보행자에게는 신호 위반의...,서울지방법원 서부지원 1993.3.9. 선고 92가합12636 판결 야간에 신호기 ...,https://drive.google.com/file/d/1Glf3M1QtrCR0k...,보행자가 녹색신호에 횡단을 개시하여 녹색신호에 충돌한 경우 보행자에게는 신호 위반의...,NaN,녹색점멸신호는 녹색신호와 동일하게 본다(도로교통법 시행규칙 별표3 참조). 이하 동...,"⊙도로교통법 제5조(신호 또는 지시에 따를 의무)\n① 도로를 통행하는 보행자, 차...",⊙도로교통법 제27조(보행자의 보호)\n① 모든 차 또는 노면전차의 운전자는 보행자...,⊙도로교통법 제48조(안전운전 및 친환경 경제운전의 의무)\n① 모든 차 또는 노면...,⊙도로교통법 시행규칙 별표 2(신호기가 표시하는 신호의 종류 및 신호의 뜻),NaN,NaN,NaN
4,보5,"신호기가 있는 횡단보도에서 적색신호에 교차로를 통과한 차량(직진, 좌회전, 우회전\...",도로교통법 제5조에 따라 보행자나 차마 등은 신호기의 신호에 따라야 할 의무가 있고...,부산지방법원 2013.11.26. 선고 2011가단17263 판결 주간에 삼거리 교...,https://drive.google.com/file/d/1W6eo0EeZS4Rvw...,도로교통법 제5조에 따라 보행자나 차마 등은 신호기의 신호에 따라야 할 의무가 있고...,① 야간 또는 주·정차된 차량 사이에서 보행자가 걸어 나오는 등 운전자가 보행자의 ...,NaN,"⊙도로교통법 제5조(신호 또는 지시에 따를 의무)\n① 도로를 통행하는 보행자, 차...",⊙도로교통법 제27조(보행자의 보호)\n① 모든 차 또는 노면전차의 운전자는 보행자...,⊙도로교통법 제48조(안전운전 및 친환경 경제운전의 의무)\n① 모든 차 또는 노면...,⊙도로교통법 시행규칙 별표 2(신호기가 표시하는 신호의 종류 및 신호의 뜻),NaN,NaN,NaN


## 2) 텍스트 추출 & 청크화

엑셀에 `case_id`, `scenario`, `explanation`, `precedents` 열이 있다고 가정합니다.
각 행을 **하나의 문서**로 보고,
scenario + explanation + precedents를 합쳐 최종 텍스트로 만든 뒤, chunking 하겠습니다.

In [6]:
all_docs = []  # 전체 Document 리스트

for idx, row in df.iterrows():
    case_id = row.get("case_id", idx)
    scenario = str(row.get("scenario", ""))
    explanation = str(row.get("explanation", ""))
    precedents = str(row.get("precedents", ""))
    reference1 = str(row.get("reference1", ""))
    reference2 = str(row.get("reference2", ""))
    reference3 = str(row.get("reference3", ""))
    reference4 = str(row.get("reference4", ""))    
    reference5 = str(row.get("reference5", ""))
    reference6 = str(row.get("reference6", ""))
    reference7 = str(row.get("reference7", ""))
    reference8 = str(row.get("reference8", ""))
    
    # 합친 텍스트
    full_text = f"[사고상황]\n{scenario}\n\n[해설]\n{explanation}\n\n[법률]\n{reference1}\n{reference2}\n{reference3}\n{reference4}\n{reference5}\n{reference6}\n{reference7}\n{reference8}".strip()
    
    # 청크화
    chunks = chunk_text(full_text, chunk_size=500, chunk_overlap=50)
    
    for chunk in chunks:
        doc = Document(
            page_content=chunk,
            metadata={
                "case_id": case_id,
                "explanation": explanation,
                "precedents": precedents,
                "reference1": reference1,
                "reference2": reference2,
                "reference3": reference3,
                "reference4": reference4,
                "reference5": reference5,
                "reference6": reference6,
                "reference7": reference7,
                "reference8": reference8
            }
        )
        all_docs.append(doc)

print(f"총 문서 청크 개수: {len(all_docs)}")
all_docs[:3]  # 미리보기

총 문서 청크 개수: 763


[Document(metadata={'case_id': '보1', 'explanation': '도로교통법 제5조에 따라 보행자나 차마는 신호기의 신호에 따라야 할 의무가 있으므로 이를 위반하여 보행자신호등 적색신호에 횡단을 개시한 보행자에게 일방적으로 과실을 물어야 할 것이나, 보행자가 상대적 교통약자인 점, 차량은 도로교통법 제27조 제1항에 따른 일시 정지 의무가 있는 점을 고려하여 보행자의 기본 과실비율을 70%로 정하였다.', 'precedents': '⊙대법원 1990.8.10. 선고 90도1116 판결 횡단보도의 표지판이나 신호대가 설치되어 있지는 않으나 도로의 바닥에 페인트로 횡단보도 표시를 하여 놓은 곳으로써 피고인이 진행하는 반대 차로 쪽은 오래되어 거의 지워진 상태이긴 하나 피고인이 운행하는 차로 쪽은 횡단보도인 점을 식별할 수 있을 만큼 그 표시가 되어있는 곳에서 교통사고가 난 경우에는 교통사고가 도로교통법상 횡단보도상에서 일어난 것으로 인정된다.\n⊙대법원 1993.2.23. 선고 92도2077 판결 차량의 운전자로서는 횡단보도의 신호가 적색인 상태에서 반대 차로 상에 정지하여 있는 차량의 뒤로 보행자가 건너오지 않을 것이라고 신뢰하는 것이 당연하고 그렇지 아니할 사태까지 예상 하여 그에 대한 주의의무를 다하여야 한다고는 할 수 없다.\n⊙서울고등법원 2002. 6. 18. 선고 2002나57692 판결 주간에 신호등이 설치되어 있는 편도2차로의 삼거리(T자) 교차로에서 B차량이 차량진행신호에 따라 직진하던 중, 좌우를 살피지 않고 보행자 정지신호에 위반하여 왕복4차로의 도로에 설치된 횡단보도를 뛰어서 건너던 A를 들이받아 상해를 입게 한 사고 : A과실 60%', 'reference1': '도로교통법 제5조에 따라 보행자나 차마는 신호기의 신호에 따라야 할 의무가 있으므로 이를 위반하여 보행자신호등 적색신호에 횡단을 개시한 보행자에게 일방적으로 과실을 물어야 할 것이나, 보행자가 상대적 교통약자인 점, 차량은 도로교통법 제27조 제1항에

## 3) 임베딩


In [8]:
embeddings = HuggingFaceEmbeddings(
    # model_name="sentence-transformers/multi-qa-mpnet-base-cos-v1"
    model_name="snunlp/KR-SBERT-V40K-klueNLI-augSTS" 
)

# persist_directory = "chroma_db_mpnet"
persist_directory = "chroma_db_krsbert"

/home/ibmuser03/.venv/lib64/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [11]:
vectorstore = Chroma.from_documents(
    documents=all_docs,
    embedding=embeddings, 
    persist_directory=persist_directory
)
vectorstore.persist()

/tmp/ipykernel_549351/2742218033.py:6: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [10]:
# vectorstore = Chroma.from_documents(
#     embedding=embeddings,  
#     persist_directory=persist_directory
# )

TypeError: Chroma.from_documents() missing 1 required positional argument: 'documents'

## 4) RAG 시연

### 4-1) DB 로드 후 Retriever 생성
로딩만 하고 싶다면, `Chroma(persist_directory, embedding_function=...)` 로 기존 DB를 불러올 수 있습니다.

In [8]:
# retriever = vectorstore.as_retriever(search_kwargs={'k': 5})
# results = retriever.invoke("횡단보도에서 보행자를 치는 사고가 난다면?")

In [9]:
# results

[Document(metadata={'case_id': '보2', 'explanation': '차량이 황색신호에 진입한 경우 신호를 위반한 것이므로 보1 기준보다 보행자의 과실을 낮추어 보행자의 기본 과실비율을 50%로 정하였다. ', 'precedents': '⊙대법원 1990.8.10. 선고 90도1116 판결\n횡단보도의 표지판이나 신호대가 설치되어 있지는 않으나 도로의 바닥에 페인트로 횡단보도\n표시를 하여 놓은 곳으로서 피고인이 진행하는 반대 차로 쪽은 오래되어 거의 지워진 상태이긴 하나\n피고인이 운행하는 차로 쪽은 횡단보도인 점을 식별할 수 있을 만큼 그 표시가 되어있는 곳에서\n교통사고가 난 경우에는 교통사고가 도로교통법상 횡단보도 상에서 일어난 것으로 인정된다.\n⊙대법원 1993.2.23. 선고 92도2077 판결\n차량의 운전자로서는 횡단보도의 신호가 적색인 상태에서 반대 차로상에 정지하여 있는 차량의\n뒤로 보행자가 건너오지 않을 것이라고 신뢰하는 것이 당연하고 그렇지 아니할 사태까지 예상\n하여 그에 대한 주의의무를 다하여야 한다고는 할 수 없다.', 'reference1': '차량이 황색신호에 진입한 경우 신호를 위반한 것이므로 보1 기준보다 보행자의 과실을 낮추어 보행자의 기본 과실비율을 50%로 정하였다. ', 'reference2': '① 야간 또는 주·정차된 차량 사이에서 보행자가 걸어 나오는 등 운전자가 보행자의 횡단을\n예상하기 어려운 경우 보행자의 과실을 5%까지 가산할 수 있다.\n② 간선도로인 경우 보행자의 횡단을 예견하기 어려우므로 보행자의 과실을 5%까지 가산할 수 있다.\n제1장. 자동차와 보행자의 사고\n③ 주택·상점가·학교는 보행자의 횡단이 잦은 곳이므로 보행자의 과실을 5%까지 감산할 수\n있다.\n④ 어린이·노인·장애인에 대한 보호를 규정한 도로교통법 제11조 내지 제12조의 2의 취지에\n비추어 차량은 항상 교통약자인 어린이·노인·장애인의 동태에 대해서 보다 주의를 기울\n여야 하는 바, 보행자가 어린이·

### WhatsonLLM

In [21]:
LLM_MODEL= "ibm/granite-3-8b-instruct"
# LLM_MODEL = "meta-llama/llama-3-3-70b-instruct"

In [22]:
# LLM 모델 준비

project_id = os.getenv("PROJECT_ID", None)
wml_credentials = {
    "apikey": os.getenv("API_KEY", None),
    "url": "https://us-south.ml.cloud.ibm.com"
}

parameters = {
    GenParams.DECODING_METHOD: DecodingMethods.GREEDY.value,
    GenParams.TOP_K: 50,       
    # GenParams.TOP_P: 0.90,       
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.MAX_NEW_TOKENS: 800,
    GenParams.STOP_SEQUENCES: ["<|endoftext|>"],
}

watsonx_llama2_korean = WatsonxLLM(
    model_id=LLM_MODEL,
    url=wml_credentials["url"],
    apikey=wml_credentials["apikey"],
    project_id=project_id,
    params=parameters
)

# RAG Retrieval
retriever = vectorstore.as_retriever(search_kwargs={'k': 5})

SYS_PROMPT = """당신은 대한민국 법률 지식을 제공하는 고급 상담 모델입니다.
당신의 역할은 아래와 같이 작동합니다:

1. RAG(Retrieval Augmented Generation) 방식으로 사용자의 질문에 답변합니다.
   - '검색 결과(context)'로 제공된 텍스트는 실제 대한민국 법률 정보이거나, 법령 해설 등으로 구성됩니다.
   - 당신은 이 '검색 결과'를 참조하되, 그 외에 임의로 사실관계를 상상하거나 지어내지 않습니다.

2. 출력 형식:
   - 먼저, 사용자의 질문에 대한 간결하고 정확한 답변을 제공합니다.
   - 만약 확실한 근거(검색 결과나 일반적 법률 상식)가 없다면, '모르겠다' 혹은 '데이터가 부족합니다'라고 답변하세요.
   - 그 후, 참고한 검색 결과(출처 or 일부 내용)나 법령 조항을 간략히 제시해줄 수 있습니다.

3. 제한사항:
   - 법률 정보는 최신성을 완전히 보장하지 않을 수 있으므로, 특정 날짜나 최신 개정 내용이 요청되면 모호하다고 답하십시오.
   - 민감한 개인정보나 변호사 자격 행사를 하지 않습니다. 단지 법률 해설이나 일반적 상담을 제공하는 역할입니다.
   - 불법, 차별적, 폭력적 사용을 조장하는 답변은 거절합니다.

4. 요약:
   - 당신은 '정직한 에이전트'로서, 법률 상담 목적으로만 답변하세요.
   - 과도한 추측이나 과장 없이, '검색결과(context)' 기반으로 답변하되, 모호하거나 근거 없으면 '모름'을 표기합니다.
"""

system_msg_template = SystemMessagePromptTemplate.from_template(SYS_PROMPT)
user_msg_template = HumanMessagePromptTemplate.from_template(
    "질문:\n{question}\n\n[검색결과]\n{context}\n---\n답변:"
)
chat_prompt = ChatPromptTemplate(
    input_variables=["question", "context"],  # 여기서 명시
    messages=[
        system_msg_template, 
        user_msg_template
    ]
)
# 6) RetrievalQA 체인 생성 (chain_type="stuff")
qa_chain = RetrievalQA.from_chain_type(
    llm=watsonx_llama2_korean,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={
        "prompt": chat_prompt
    }
)


In [23]:
question = "자동차를 타고 황색불에 교차로로 진입하였는데 횡단보도를 건너는 보행자와 사고가 났어"
result = qa_chain({"query": question})  # stuff 체인 내부에서 question-> prompt

print("\n[User query]", question)
print("[Assistant answer]\n", result["result"])  # 최종 답변
print("[Source docs used]:", result["source_documents"])  # RAG에서 사용된 chunks


[User query] 자동차를 타고 황색불에 교차로로 진입하였는데 횡단보도를 건너는 보행자와 사고가 났어
[Assistant answer]
 
사고 상황에 따라 달라질 수 있습니다.

1. 신호기가 있는 교차로에서 녹색신호에 교차로 및 횡단보도를 직진 또는 좌회전으로 통과한 차량이 보행자신호등 적색신호에 횡단보도 부근을 건너고 있던 보행자를 충격한 사고:
   - 답변: 차량의 과실이 크므로 보행자의 기본 과실비율을 50%로 정합니다.

2. 보도와 차도의 구분이 있는 도로에서 차도를 진행하는 차량이 “차도가 아닌 장소”로 진출 또는 “차도가 아닌 장소”에서 차도로 진입하다가 보도 위를 걷고 있는 보행자를 충격한 사고:
   - 답변: 차량의 일방과실로 정하며, 차량의 과실이 100%입니다.

3. 신호기가 있는 횡단보도를 통과하여 교차로(단일로를 포함하며, 직선로나 곡선로인지 불문)에 진입하려는 차량이 보행자신호등 적색신호에 횡단보도를 건너고 있던 보행자를 보행자신호등 녹색신호로 바뀐 상황에서 충격한 사고:
   - 답변: 보행자의 기본 과실비율을 20%로 정합니다.

참고:
- 위의 사고 상황과 해설은 대한민국 도로교통법과 관련 법률에 기반한 것입니다.
- 사고 상황에 대한 정확한 정보가 부족하거나, 특정 날짜나 최신 개정 내용이 요청되면 모호하다고 답변합니다.
- 민감한 개인정보나 변호사 자격 행사를 하지 않습니다. 단지 법률 해설이나 일반적 상담을 제공하는 역할입니다.
- 불법, 차별적, 폭력적 사용을 조장하는 답변은 거절합니다.
[Source docs used]: [Document(metadata={'case_id': '보16', 'explanation': '차량이 신호를 준수하고 보행자가 보행자신호를 위반하여 횡단보도 부근에서 횡단한\n경우에는 보행자의 과실이 크므로 보행자의 기본 과실비율을 50%로 정하였다.', 'precedents': '◆서울민사지방법원 1994.3.22.93가단24542 판결 야간에 편도4차로의 도로에서 B차량이 2차로를 

위에서 `question`을 변경해서 질문해볼 수도 있고,
LLM을 WatsonxLLM으로 교체하거나,
임베딩 모델을 다르게 교체할 수도 있습니다.